In [362]:
import numpy as np
from numpy import random
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
import sys 
import shutil
from sklearn.metrics import mean_squared_error
from datetime import datetime
from pandas import read_csv
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from scipy.signal import find_peaks

In [363]:
#Data location
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/OFF_data/"

In [364]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)     

In [365]:
# Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [366]:
#Function to obtain my custom features

import warnings
warnings.filterwarnings("ignore")
def peak_study(array,time,delta_array):
    
    indices = find_peaks(array)
    indices=np.delete(indices,-1)
    y=[array[j] for j in indices]
    x=[time[j] for j in indices]
    y_err=[delta_array[j] for j in indices]
    
    x=np.hstack(x)
    y=np.hstack(y)
    y_err=np.hstack(y_err)
    
    arr=[1,-1]
    for i in range(len(y)):
        if np.isnan(y[i])==False:
            error=y_err[i]
            choice=np.random.choice(arr,1)
            if choice ==1:
                y[i]=y[i]+error
            if choice==-1:
                y[i]=y[i]-error
                
    peak_magnitudes=y
    nb_peaks=len(y)
    delta_energy_arr=[]
    delta_time_arr=[]
    
    for i in range(nb_peaks-1):
        
        delta = x[i+1]-x[i]
        delta_energy=abs(y[i+1]-y[i])
        delta_time_arr.append(delta)
        delta_energy_arr.append(delta_energy)

        
    NG_diff_mean_peaks=np.mean(delta_energy_arr)
    max_diff_mean_peaks=max(y)-np.mean(y)
    min_diff_mean_peaks=min(y)-np.mean(y)
    peaks_time_delay=np.mean(delta_time_arr)
    std_peaks=np.std(y)
    mean_peaks=np.mean(y)
    var_peaks=np.var(y)
    mean_arr=np.mean(array)
    maxi=max(array)
    mini=min(array)
    amplitude=max(array)-min(array)
    return nb_peaks,peak_magnitudes,peaks_time_delay,std_peaks,mean_peaks,NG_diff_mean_peaks,max_diff_mean_peaks,min_diff_mean_peaks,mean_arr,maxi,mini,amplitude


In [367]:

nb_peaksON=[]
peak_magnitudesON=[]
peaks_time_delayON=[]
std_peaksON=[]
mean_peaksON=[]

NG_diff_mean_peaksON=[]
max_diff_mean_peaksON=[]
min_diff_mean_peaksON=[]
for i in range(len(dataON)):
    
    dataframe=pd.read_csv(dataON[i],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']
    
    mjd=np.array(dataframe['MJD'])
    flux=np.array(dataframe['Flux'])
    photon_idx=np.array(dataframe['Photon Index'])
    delta_index=np.array(dataframe['Delta Index'])
    
    nb_peaks,peak_magnitudes,peaks_time_delay,std_peaks,mean_peaks,NG_diff_mean_peaks,max_diff_mean_peaks,min_diff_mean_peaks = peak_study(flux,mjd)
    
    nb_peaksON.append(nb_peaks)
    
    peak_magnitudesON.append(peak_magnitudes)
    peaks_time_delayON.append(peaks_time_delay)
    std_peaksON.append(std_peaks)
    mean_peaksON.append(mean_peaks)

    NG_diff_mean_peaksON.append(NG_diff_mean_peaks)
    max_diff_mean_peaksON.append(max_diff_mean_peaks)
    min_diff_mean_peaksON.append(min_diff_mean_peaks)
    
print("Mean Number of  peaks: ",np.mean(nb_peaksON))
print("Average magnitude of peaks :  ",np.mean(mean_peaksON))
print("Average delay between peaks: ",np.mean(peaks_time_delayON))
print("Average difference between next neighbour peak magnitudes :  ",np.mean(NG_diff_mean_peaksON))
print("Average difference between biggest peak and mean peak magnitude :  ",np.mean(max_diff_mean_peaksON))
print("Average difference between minimum peak and mean peak magnitude :  ",np.mean(min_diff_mean_peaksON))
print("Average standard deviation of peaks :  ",np.mean(std_peaksON))

TypeError: peak_study() missing 1 required positional argument: 'delta_array'

In [ ]:
nb_peaksOFF=[]
peak_magnitudesOFF=[]
peaks_time_delayOFF=[]
std_peaksOFF=[]
mean_peaksOFF=[]

NG_diff_mean_peaksOFF=[]
max_diff_mean_peaksOFF=[]
min_diff_mean_peaksOFF=[]
for i in range(len(dataOFF)):
    
    dataframe=pd.read_csv(dataOFF[i],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','PhotOFF Index','Delta Index','TS','fratio']
    
    mjd=np.array(dataframe['MJD'])
    flux=np.array(dataframe['Flux'])
    photOFF_idx=np.array(dataframe['PhotOFF Index'])
    delta_index=np.array(dataframe['Delta Index'])
    
    nb_peaks,peak_magnitudes,peaks_time_delay,std_peaks,mean_peaks,NG_diff_mean_peaks,max_diff_mean_peaks,min_diff_mean_peaks = peak_study(flux,mjd)
    
    nb_peaksOFF.append(nb_peaks)
    
    peak_magnitudesOFF.append(peak_magnitudes)
    peaks_time_delayOFF.append(peaks_time_delay)
    std_peaksOFF.append(std_peaks)
    mean_peaksOFF.append(mean_peaks)

    NG_diff_mean_peaksOFF.append(NG_diff_mean_peaks)
    max_diff_mean_peaksOFF.append(max_diff_mean_peaks)
    min_diff_mean_peaksOFF.append(min_diff_mean_peaks)
    
print("Mean Number of  peaks: ",np.mean(nb_peaksOFF))
print("Average magnitude of peaks :  ",np.mean(mean_peaksOFF))
print("Average delay between peaks: ",np.mean(peaks_time_delayOFF))
print("Average difference between next neighbour peak magnitudes :  ",np.mean(NG_diff_mean_peaksOFF))
print("Average difference between biggest peak and mean peak magnitude :  ",np.mean(max_diff_mean_peaksOFF))
print("Average difference between minimum peak and mean peak magnitude :  ",np.mean(min_diff_mean_peaksOFF))
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
print("Average standard deviation of peaks :  ",np.mean(std_peaksOFF))   

In [281]:
#Machine learning section and testing

In [369]:
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF

Labels=[]

nb_peaks_arr_photon =[]
peak_magnitudes_arr_photon =[]
peaks_time_delay_arr_photon =[]
std_peaks_arr_photon =[]
mean_peaks_arr_photon =[]

NG_diff_mean_peaks_arr_photon =[]
max_diff_mean_peaks_arr_photon =[]
min_diff_mean_peaks_arr_photon =[]

nb_peaks_arr_flux =[]
peak_magnitudes_arr_flux =[]
peaks_time_delay_arr_flux =[]
std_peaks_arr_flux =[]
mean_peaks_arr_flux =[]

NG_diff_mean_peaks_arr_flux =[]
max_diff_mean_peaks_arr_flux =[]
min_diff_mean_peaks_arr_flux =[]



for i in range(len(dataOFF)):
    
    dataframe=pd.read_csv(dataOFF[i],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']
    
    mjd=np.array(dataframe['MJD'])
    flux=np.array(dataframe['Flux'])
    photon_idx=np.array(dataframe['Photon Index'])
    delta_index=np.array(dataframe['Delta Index'])
    delta_flux=np.array(dataframe['Delta Flux'])
    
    nb_peaks,peak_magnitudes,peaks_time_delay,std_peaks,
    mean_peaks,NG_diff_mean_peaks,max_diff_mean_peaks,min_diff_mean_peaks,mean_arr,maxi,mini,amplitude= peak_study(photon_idx,mjd,delta_index)
    
    nb_peaks_arr_photon.append(nb_peaks)
    peak_magnitudes_arr_photon.append(peak_magnitudes)
    peaks_time_delay_arr_photon.append(peaks_time_delay)
    std_peaks_arr_photon.append(std_peaks)
    mean_peaks_arr_photon.append(mean_peaks)
    NG_diff_mean_peaks_arr_photon.append(NG_diff_mean_peaks)
    max_diff_mean_peaks_arr_photon.append(max_diff_mean_peaks)
    min_diff_mean_peaks_arr_photon.append(min_diff_mean_peaks)
    nb_peaks,peak_magnitudes,peaks_time_delay,std_peaks,mean_peaks,NG_diff_mean_peaks,max_diff_mean_peaks,min_diff_mean_peaks = peak_study(flux,mjd,delta_flux)

                                                                                                                                           
    nb_peaks_arr_flux.append(nb_peaks)
    peak_magnitudes_arr_flux.append(peak_magnitudes)
    peaks_time_delay_arr_flux.append(peaks_time_delay)
    std_peaks_arr_flux.append(std_peaks)
    mean_peaks_arr_flux.append(mean_peaks)

    NG_diff_mean_peaks_arr_flux.append(NG_diff_mean_peaks)
    max_diff_mean_peaks_arr_flux.append(max_diff_mean_peaks)
    min_diff_mean_peaks_arr_flux.append(min_diff_mean_peaks)
for i in range(len(dataON)):
    
    dataframe=pd.read_csv(dataON[i],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']
    
    mjd=np.array(dataframe['MJD'])
    flux=np.array(dataframe['Flux'])
    photon_idx=np.array(dataframe['Photon Index'])
    delta_index=np.array(dataframe['Delta Index'])
    delta_flux=np.array(dataframe['Delta Flux'])
    
    nb_peaks,peak_magnitudes,peaks_time_delay,std_peaks,mean_peaks,NG_diff_mean_peaks,max_diff_mean_peaks,min_diff_mean_peaks = peak_study(photon_idx,mjd,delta_index)
    
    nb_peaks_arr_photon.append(nb_peaks)
    peak_magnitudes_arr_photon.append(peak_magnitudes)
    peaks_time_delay_arr_photon.append(peaks_time_delay)
    std_peaks_arr_photon.append(std_peaks)
    mean_peaks_arr_photon.append(mean_peaks)

    NG_diff_mean_peaks_arr_photon.append(NG_diff_mean_peaks)
    max_diff_mean_peaks_arr_photon.append(max_diff_mean_peaks)
    min_diff_mean_peaks_arr_photon.append(min_diff_mean_peaks)

    nb_peaks,peak_magnitudes,peaks_time_delay,std_peaks,mean_peaks,NG_diff_mean_peaks,max_diff_mean_peaks,min_diff_mean_peaks = peak_study(flux,mjd,delta_flux)

    nb_peaks_arr_flux.append(nb_peaks)
    peak_magnitudes_arr_flux.append(peak_magnitudes)
    peaks_time_delay_arr_flux.append(peaks_time_delay)
    std_peaks_arr_flux.append(std_peaks)
    mean_peaks_arr_flux.append(mean_peaks)

    NG_diff_mean_peaks_arr_flux.append(NG_diff_mean_peaks)
    max_diff_mean_peaks_arr_flux.append(max_diff_mean_peaks)
    min_diff_mean_peaks_arr_flux.append(min_diff_mean_peaks)

#Creating labels
for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    Labels.append(int(1))

In [22]:
#Best performance

nbfeatures=14

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_photon
data_matrix[1]=peaks_time_delay_arr_photon
data_matrix[2]=std_peaks_arr_photon
data_matrix[3]=mean_peaks_arr_photon
data_matrix[4]=NG_diff_mean_peaks_arr_photon
data_matrix[5]=max_diff_mean_peaks_arr_photon
data_matrix[6]=min_diff_mean_peaks_arr_photon

data_matrix[7]=nb_peaks_arr_flux
data_matrix[8]=peaks_time_delay_arr_flux
data_matrix[9]=std_peaks_arr_flux
data_matrix[10]=mean_peaks_arr_flux
data_matrix[11]=NG_diff_mean_peaks_arr_flux
data_matrix[12]=max_diff_mean_peaks_arr_flux
data_matrix[13]=min_diff_mean_peaks_arr_flux


data_matrix=data_matrix.reshape((lg,nbfeatures))

In [176]:
import cesium
from cesium import featurize

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
bad_ON=[]
iterations=300
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

for i in  range(iterations):
                 
    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.1, random_state=i)

    model = RandomForestClassifier(n_estimators=1000, max_features="auto",
                                          random_state=0,class_weight =class_weight)
    Labels=np.hstack(Labels)
    model.fit(x_train, y_train)
    prediction= model.predict(x_test)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0

    for i in range(len(y_test)):
        if y_test[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
                
            else : 
                fon+=1
                bad_ON.append(i)
        if y_test[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(prediction,y_test,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))

Accuracy for ON class:  94.53414195453668 %
Accuracy for OFF class:  73.59036749269708 %
False Positive rate:  5.465858045463309 %
False Negative rate:  26.40963250730291 %
F1 score:  0.6982718400447031


In [23]:
#Best performance

nbfeatures=14

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_photon
data_matrix[1]=peaks_time_delay_arr_photon
data_matrix[2]=std_peaks_arr_photon
data_matrix[3]=mean_peaks_arr_photon
data_matrix[4]=NG_diff_mean_peaks_arr_photon
data_matrix[5]=max_diff_mean_peaks_arr_photon
data_matrix[6]=min_diff_mean_peaks_arr_photon

data_matrix[7]=nb_peaks_arr_flux
data_matrix[8]=peaks_time_delay_arr_flux
data_matrix[9]=std_peaks_arr_flux
data_matrix[10]=mean_peaks_arr_flux
data_matrix[11]=NG_diff_mean_peaks_arr_flux
data_matrix[12]=max_diff_mean_peaks_arr_fux
data_matrix[13]=min_diff_mean_peaks_arr_flux


data_matrix=data_matrix.reshape((lg,nbfeatures))

In [37]:
#Best performance

nbfeatures=10

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_photon
data_matrix[1]=peaks_time_delay_arr_photon
data_matrix[2]=std_peaks_arr_photon
data_matrix[3]=mean_peaks_arr_photon

data_matrix[4]=nb_peaks_arr_flux
data_matrix[5]=peaks_time_delay_arr_flux
data_matrix[6]=std_peaks_arr_flux
data_matrix[7]=mean_peaks_arr_flux

data_matrix[8]=max_diff_mean_peaks_arr_photon
data_matrix[9]=min_diff_mean_peaks_arr_photon


data_matrix=data_matrix.reshape((lg,nbfeatures))

In [373]:
import cesium
from cesium import featurize

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
bad_ON=[]
iterations=10
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

for i in  range(iterations):
                 
    x_train, x_test, y_train, y_test = train_test_split(data_matrix, Labels, test_size=0.1, random_state=i)

    model = RandomForestClassifier(n_estimators=2000, max_features="auto",
                                          random_state=0,class_weight =class_weight)
    Labels=np.hstack(Labels)
    model.fit(x_train, y_train)
    prediction= model.predict(x_test)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0

    for i in range(len(y_test)):
        if y_test[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
                
            else : 
                fon+=1
                bad_ON.append(i)
        if y_test[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(prediction,y_test,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))

Accuracy for ON class:  100.0 %
Accuracy for OFF class:  51.87486650353194 %
False Positive rate:  0.0 %
False Negative rate:  48.12513349646805 %
F1 score:  0.41372865185372165


In [102]:
nbfeatures=1

data_matrix=np.zeros((nbfeatures,lg))


# data_matrix[0]=NG_diff_mean_peaks_arr_flux
# Accuracy for ON class:  26.75 %
# Accuracy for OFF class:  70.09345794392524 %

# data_matrix[0]=nb_peaks_arr_flux
# Accuracy for ON class:  31.963203463203456 %
# Accuracy for OFF class:  67.50933126425241 %

# data_matrix[0]=peaks_time_delay_arr_flux
# Accuracy for ON class:  27.4025974025974 %
# Accuracy for OFF class:  80.40975514402771 %

# data_matrix[0]=std_peaks_arr_flux
# Accuracy for ON class:  26.75 %
# Accuracy for OFF class:  70.27782586117362 %

# data_matrix[0]=mean_peaks_arr_flux
# Accuracy for ON class:  26.75 %
# Accuracy for OFF class:  69.71333490347962 %

# data_matrix[0]=max_diff_mean_peaks_arr_flux
# Accuracy for ON class:  8.0 %
# Accuracy for OFF class:  90.08198813300353 %

# data_matrix[0]=min_diff_mean_peaks_arr_flux
# Accuracy for ON class:  30.909090909090914 %
# Accuracy for OFF class:  70.18348623853211 %

# data_matrix[0]=nb_peaks_arr_photon
# Accuracy for ON class:  48.1017316017316 %
# Accuracy for OFF class:  48.44518257846103 %

# data_matrix[0]=peaks_time_delay_arr_photon
# Accuracy for ON class:  49.740620490620486 %
# Accuracy for OFF class:  59.03121287237995 %

# data_matrix[0]=std_peaks_arr_photon
# Accuracy for ON class:  9.31060606060606 %
# Accuracy for OFF class:  90.68022215603669 %

# data_matrix[0]=mean_peaks_arr_photon
# Accuracy for ON class:  14.19047619047619 %
# Accuracy for OFF class:  92.45828502016062 %


# data_matrix[0]=max_diff_mean_peaks_arr_photon
# Accuracy for ON class:  3.7272727272727266 %
# Accuracy for OFF class:  91.04562696945551 %

# data_matrix[0]=min_diff_mean_peaks_arr_photon
# Accuracy for ON class:  8.163419913419911 %
# Accuracy for OFF class:  90.27475417186 %

# data_matrix[0]=NG_diff_mean_peaks_arr_photon
# Accuracy for ON class:  1.7424242424242422 %
# Accuracy for OFF class:  92.46567222305951 %
    
data_matrix=data_matrix.reshape((lg,nbfeatures))

In [88]:
nbfeatures=5

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=min_diff_mean_peaks_arr_flux
data_matrix[1]=peaks_time_delay_arr_flux
data_matrix[2]=nb_peaks_arr_flux
data_matrix[3]=std_peaks_arr_flux
data_matrix=data_matrix.reshape((lg,nbfeatures))

In [ ]:


nbfeatures=8

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=std_peaks_arr_photon
data_matrix[1]=NG_diff_mean_peaks_arr_photon
data_matrix[2]=max_diff_mean_peaks_arr_photon
data_matrix[3]=min_diff_mean_peaks_arr_photon

data_matrix[4]=std_peaks_arr_flux
data_matrix[5]=NG_diff_mean_peaks_arr_flux
data_matrix[6]=max_diff_mean_peaks_arr_flux
data_matrix[7]=min_diff_mean_peaks_arr_flux


data_matrix=data_matrix.reshape((lg,nbfeatures))

# 10 MC

# Accuracy for ON class:  100.0 %
# Accuracy for OFF class:  56.868194814724596 %
# False Positive rate:  0.0 %
# False Negative rate:  43.131805185275404 %
# F1 score:  0.5168033220908891

# 300 MC steps

# Accuracy for ON class:  99.97222222222221 %
# Accuracy for OFF class:  58.16069649236325 %
# False Positive rate:  0.027777777777777773 %
# False Negative rate:  41.839303507636735 %
# F1 score:  0.5325707055521958

In [36]:


nbfeatures=8

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_photon
data_matrix[1]=peaks_time_delay_arr_photon
data_matrix[2]=std_peaks_arr_photon
data_matrix[3]=mean_peaks_arr_photon

data_matrix[4]=nb_peaks_arr_flux
data_matrix[5]=peaks_time_delay_arr_flux
data_matrix[6]=std_peaks_arr_flux
data_matrix[7]=mean_peaks_arr_flux




data_matrix=data_matrix.reshape((lg,nbfeatures))

# Accuracy for ON class:  94.93181818181817 %
# Accuracy for OFF class:  82.54595549480776 %
# False Positive rate:  5.068181818181818 %
# False Negative rate:  17.454044505192243 %
# F1 score:  0.8023587748899297

In [ ]:
nbfeatures=10

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_photon
data_matrix[1]=peaks_time_delay_arr_photon
data_matrix[2]=std_peaks_arr_photon
data_matrix[3]=mean_peaks_arr_photon

data_matrix[4]=nb_peaks_arr_flux
data_matrix[5]=peaks_time_delay_arr_flux
data_matrix[6]=std_peaks_arr_flux
data_matrix[7]=mean_peaks_arr_flux

data_matrix[8]=NG_diff_mean_peaks_arr_photon
data_matrix[9]=NG_diff_mean_peaks_arr_flux


data_matrix=data_matrix.reshape((lg,nbfeatures))

# Accuracy for ON class:  89.80122655122655 %
# Accuracy for OFF class:  77.222579721542 %
# False Positive rate:  10.198773448773448 %
# False Negative rate:  22.777420278458003 %
# F1 score:  0.7353412123485507

In [ ]:

nbfeatures=12

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_photon
data_matrix[1]=peaks_time_delay_arr_photon
data_matrix[2]=std_peaks_arr_photon
data_matrix[3]=mean_peaks_arr_photon

data_matrix[4]=nb_peaks_arr_flux
data_matrix[5]=peaks_time_delay_arr_flux
data_matrix[6]=std_peaks_arr_flux
data_matrix[7]=mean_peaks_arr_flux

data_matrix[8]=max_diff_mean_peaks_arr_photon
data_matrix[9]=min_diff_mean_peaks_arr_photon

data_matrix[10]=max_diff_mean_peaks_arr_flux
data_matrix[11]=min_diff_mean_peaks_arr_flux

data_matrix=data_matrix.reshape((lg,nbfeatures))

# Accuracy for ON class:  100.0 %
# Accuracy for OFF class:  74.90497932413209 %
# False Positive rate:  0.0 %
# False Negative rate:  25.09502067586792 %
# F1 score:  0.7175846530763869

In [370]:
#Best performance ?

nbfeatures=10

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_photon
data_matrix[1]=peaks_time_delay_arr_photon
data_matrix[2]=std_peaks_arr_photon
data_matrix[3]=mean_peaks_arr_photon

data_matrix[4]=nb_peaks_arr_flux
data_matrix[5]=peaks_time_delay_arr_flux
data_matrix[6]=std_peaks_arr_flux
data_matrix[7]=mean_peaks_arr_flux

data_matrix[8]=max_diff_mean_peaks_arr_photon
data_matrix[9]=min_diff_mean_peaks_arr_photon


data_matrix=data_matrix.reshape((lg,nbfeatures))

# 10 MC steps

# Accuracy for ON class:  95.95959595959596 %
# Accuracy for OFF class:  98.38054141266277 %
# False Positive rate:  4.04040404040404 %
# False Negative rate:  1.6194585873372422 %
# F1 score:  0.9810545428449524


# 300 MC steps

# Accuracy for ON class:  94.97461664152841 %
# Accuracy for OFF class:  97.9965918395496 %
# False Positive rate:  5.0253833584715935 %
# False Negative rate:  2.0034081604504066 %
# F1 score:  0.9761534399258989

In [45]:

nbfeatures=6

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_flux
data_matrix[1]=peaks_time_delay_arr_flux
data_matrix[2]=std_peaks_arr_flux
data_matrix[3]=mean_peaks_arr_flux

data_matrix[4]=max_diff_mean_peaks_arr_flux
data_matrix[5]=min_diff_mean_peaks_arr_flux


data_matrix=data_matrix.reshape((lg,nbfeatures))

# 30 MC steps : 

# Accuracy for ON class:  98.9469696969697 %
# Accuracy for OFF class:  40.384589109757464 %
# False Positive rate:  1.0530303030303032 %
# False Negative rate:  59.61541089024253 %
# F1 score:  0.3578440452228621

In [372]:

nbfeatures=4

data_matrix=np.zeros((nbfeatures,lg))

data_matrix[0]=nb_peaks_arr_flux
data_matrix[1]=peaks_time_delay_arr_flux
data_matrix[2]=std_peaks_arr_flux
data_matrix[3]=mean_peaks_arr_flux

# data_matrix[4]=max_diff_mean_peaks_arr_flux
# data_matrix[5]=min_diff_mean_peaks_arr_flux


data_matrix=data_matrix.reshape((lg,nbfeatures))

# Accuracy for ON class:  93.3388370888371 %
# Accuracy for OFF class:  55.171118790298614 %
# False Positive rate:  6.661162911162912 %
# False Negative rate:  44.828881209701386 %
# F1 score:  0.4926499253078917

In [351]:
data=pd.read_csv(dataON[5],index_col=[0])
dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

mjd=np.array(dataframe['MJD'])
flux=np.array(dataframe['Flux'])
photon_idx=np.array(dataframe['Photon Index'])
delta_index=np.array(dataframe['Delta Index'])
delta_flux=np.array(dataframe['Delta Flux'])

print(len(delta_index))

37
